In [19]:
import psycopg2
import pyarrow as pa
import pyarrow.parquet as pq
import os
import pandas as pd
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import col
from pyspark.sql.functions import col, concat_ws, lit, substring_index
import matplotlib.pyplot as plt
from difflib import SequenceMatcher


# Create a Spark session
spark = SparkSession.builder.appName("ReadParquetExample").getOrCreate()

from delta import *

# Configurações de conexão com o banco de dados
db_params = {
    'host': 'psql-mock-database-cloud.postgres.database.azure.com',
    'port': '5432',
    'dbname': 'ecom1692637467959vmhtnfhkniqcdrvy',
    'user': 'sblzxqbkzxogfxlutfgnyycw@psql-mock-database-cloud',
    'password': 'qvcedokxqkibwxtcclvgfmte'
}

# JDBC URL for PostgreSQL
jdbc_url = f"jdbc:postgresql://{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# JDBC connection properties
properties = {
    'user': db_params['user'],
    'password': db_params['password'],
    'driver': 'org.postgresql.Driver'
}

pasta_base = 'delta'

try:
    os.mkdir(pasta_base)
except:
    shutil.rmtree(pasta_base)
    os.mkdir(pasta_base)




In [20]:
# Conecta ao banco de dados
conn = psycopg2.connect(**db_params)

# Cria um cursor
cursor = conn.cursor()

# Executa a consulta para listar as tabelas
query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'"
cursor.execute(query)

# Obtém os resultados
tabelas = [row[0] for row in cursor.fetchall()]
print(tabelas)
# Fecha o cursor
cursor.close()

['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'product_lines', 'products', 'pg_stat_statements', 'pg_buffercache']


In [21]:
# Cria uma pasta para os arquivos Parquet (se não existir)
pasta_base = 'delta/'
tabelas_suc = []
# Itera sobre as tabelas e salva cada uma em uma pasta separada
for tabela in tabelas:
    try:
        query_tabela = f"SELECT * FROM {tabela}"
        df = pd.read_sql(query_tabela, conn)
        table = pa.Table.from_pandas(df)

        # Cria uma pasta para a tabela (se não existir)
        pasta_tabela = os.path.join(pasta_base, tabela)
        if not os.path.exists(pasta_tabela):
            os.makedirs(pasta_tabela)

        caminho_parquet = os.path.join(pasta_tabela, f'{tabela}.parquet')
        pq.write_table(table, caminho_parquet)
        print(f'\n *** Tabela {tabela} salva como Parquet em {caminho_parquet} ***')
        tabelas_suc.append(tabela)
    except:
        print(f'\n *** Tabela {tabela} erro ao salvar como Parquet em {caminho_parquet} ***')

    #print(f'\n *** Tabela {tabela} não salva como Delta ***')

/tmp/ipykernel_850977/1049969471.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_tabela, conn)



 *** Tabela customers salva como Parquet em delta/customers/customers.parquet ***

 *** Tabela employees salva como Parquet em delta/employees/employees.parquet ***


/tmp/ipykernel_850977/1049969471.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_tabela, conn)
/tmp/ipykernel_850977/1049969471.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_tabela, conn)



 *** Tabela offices salva como Parquet em delta/offices/offices.parquet ***


/tmp/ipykernel_850977/1049969471.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_tabela, conn)


...


 *** Tabela products salva como Parquet em delta/products/products.parquet ***


/tmp/ipykernel_850977/1049969471.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_tabela, conn)



 *** Tabela pg_stat_statements salva como Parquet em delta/pg_stat_statements/pg_stat_statements.parquet ***


/tmp/ipykernel_850977/1049969471.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_tabela, conn)



 *** Tabela pg_buffercache erro ao salvar como Parquet em delta/pg_stat_statements/pg_stat_statements.parquet ***


In [24]:
parquet_files = []

print(tabelas_suc)
for tabela in tabelas_suc:
    pasta_tabela = os.path.join(pasta_base, tabela)
    caminho_parquet = os.path.join(pasta_tabela, f'{tabela}.parquet')
    parquet_files.append(caminho_parquet)

dataframes = [pd.read_parquet(file) for file in parquet_files]

# Mapear tipos de dados Pandas para tipos DBML
pandas_to_dbml_types = {
    'int64': 'integer',
    'float64': 'float',
    'object': 'varchar',
    'datetime64': 'timestamp',
    'bool': 'boolean',
}

# Crie o código DBML a partir dos DataFrames
pandas_to_dbml_types = {
    'int64': 'integer',
    'float64': 'float',
    'object': 'varchar',
    'datetime64': 'timestamp',
    'bool': 'boolean',
}

# Crie o código DBML a partir dos DataFrames
dbml_code = ""
tables = []

for idx, df in enumerate(dataframes):
    table_name = tabelas_suc[idx]
    table_code = f"Table {table_name} {{\n"
    columns = []

    for col, dtype in df.dtypes.items():
        pandas_type = str(dtype)
        dbml_type = pandas_to_dbml_types.get(pandas_type, 'varchar')
        columns.append((col, dbml_type))

    table_code += f"  {columns[0][0]} {columns[0][1]} [primary key]\n"
    for col_name, col_type in columns[1:]:
        table_code += f"  {col_name} {col_type}\n"

    table_code += "}\n\n"
    dbml_code += table_code
    tables.append((table_name, columns))

# Gerar relações automaticamente com base nas semelhanças de nomes
for i, (table_name, columns) in enumerate(tables):
    for j, (other_table_name, other_columns) in enumerate(tables):
        if i != j:
            for col_name, _ in columns:
                for other_col_name, _ in other_columns:
                    if col_name == other_col_name:
                        dbml_code += f"Ref: {table_name}.{col_name} < {other_table_name}.{other_col_name}\n"
                        dbml_code += f"Ref: {other_table_name}.{other_col_name} < {table_name}.{col_name}\n"

# Salvar o código DBML em um arquivo
with open('database.dbml', 'w') as dbml_file:
    dbml_file.write(dbml_code)

print("Código DBML gerado e salvo com sucesso.")

['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'product_lines', 'products', 'pg_stat_statements']
Código DBML gerado e salvo com sucesso.


Qual país possui a maior quantidade de itens cancelados?

In [ ]:
# Pasta customer
parquet_file_path = 'delta/customers/customers.parquet'
df_customer = spark.read.parquet(parquet_file_path)

# Pasta orders
parquet_file_path = 'delta/orders/orders.parquet'
df_orders = spark.read.parquet(parquet_file_path)

# Merge
df_customer_orders = df_orders.join(df_customer, on='customer_number', how='left')

df_customer_orders_cancel = df_customer_orders[df_customer_orders['status']=='Cancelled']
country_value_counts = df_customer_orders_cancel.groupBy('country').count()
country_value_counts_sorted = country_value_counts.orderBy(desc('count'))
country_value_counts_sorted.show()
first_country_name = country_value_counts_sorted.select('country').first()[0]

print("--------------------------------------------------------")
print("Venda mais realizada:", first_country_name)
print("--------------------------------------------------------")



Qual o faturamento da linha de produto mais vendido, considere como os itens Shipped, cujo o pedido foi realizado no ano de 2005?

In [ ]:
# Pasta orderdetails
parquet_file_path = 'delta/orderdetails/orderdetails.parquet'
df_orderdetails = spark.read.parquet(parquet_file_path)

# Pasta orders
parquet_file_path = 'delta/orders/orders.parquet'
df_orders = spark.read.parquet(parquet_file_path)
df_orders = df_orders[df_orders['status']=='Shipped']
df_orders = df_orders[df_orders['order_date'] >= '2005-01-01']
df_orders = df_orders[df_orders['order_date'] < '2006-01-01']

# Pasta products
parquet_file_path = 'delta/products/products.parquet'
df_products = spark.read.parquet(parquet_file_path)
print(df_products.columns)

# Merge
df_order_orderdetails = df_orders.join(df_orderdetails, on='order_number', how='left')
df_order_orderdetails = df_order_orderdetails.join(df_products[['product_name','product_code','buy_price']], on='product_code', how='left')
print(df_order_orderdetails.columns)

product_value_counts = df_order_orderdetails.groupBy('product_name').count()
product_value_counts_sorted = product_value_counts.orderBy(desc('count'))

first_product_name = product_value_counts_sorted.select('product_name').first()[0]
df_order_orderdetails = df_order_orderdetails[df_order_orderdetails['product_name']==first_product_name]

df_with_product = df_order_orderdetails.withColumn("product_total", col("quantity_ordered") * col("price_each"))
total_buy_price = df_with_product.agg({"product_total": "sum"}).collect()[0][0]

print("--------------------------------------------------------")
print("Venda mais realizada:", first_product_name)
print("Total:", total_buy_price)
print("--------------------------------------------------------")


Nome, sobrenome e e-mail dos vendedores do Japão, o local-part do e-mail deve estar mascarado

In [ ]:
# Pasta employees
parquet_file_path = 'delta/employees/employees.parquet'
df_employees = spark.read.parquet(parquet_file_path)

# Pasta employees
parquet_file_path = 'delta/offices/offices.parquet'
df_offices = spark.read.parquet(parquet_file_path)
df_employees_offices = df_employees.join(df_offices, on='office_code', how='left')
df_vendedores_japao = df_employees_offices.filter(col("country") == "Japan")
masked_email = concat_ws('@', substring_index(col("email"), '@', 1), lit("*****"))
df_resultado = df_vendedores_japao.select("first_name", "last_name", masked_email.alias("masked_email"))
df_resultado.show()



In [ ]:
spark.stop()